In [1]:
# pyfredapi need API Key which can be applied from below website
# https://fred.stlouisfed.org/docs/api/api_key.html

# install api for access financial data from FRED website https://fred.stlouisfed.org/
!pip install pyfredapi --upgrade --quiet
# install api for access google bigquery
!pip install google-cloud-bigquery --quiet

In [2]:
# import api for access financial data 
import pyfredapi as pf
# import api for access google bigquery
from google.cloud import bigquery as bq
# import for getting details for authentication and project id
import google.auth

In [3]:
df = pf.get_series('FEDFUNDS')[['date','value']].set_index('date').rename(columns={'value':'fed_rate'})

In [4]:
df

,fed_rate
date,
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83
...,...
2024-05-01,5.33
2024-06-01,5.33
2024-07-01,5.33


In [5]:
# key is generated at Service Account and downloaded in json format
# default credentials is set under environment variable "GOOGLE_APPLICATION_CREDENTIALS"
# the value of "GOOGLE_APPLICATION_CREDENTIALS" is set to path of json e.g. '/Users/.../project_id.json'

# get project_id from json
credentials, project_id = google.auth.default()
database_id = 'data'
table_id = 'fed_rate'

In [6]:
df.to_gbq(f'{database_id}.{table_id}', project_id=project_id, if_exists='replace')

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 902.58it/s]


In [7]:
# Check if data can be accessed
client = bq.Client()
sql = f'select * from {project_id}.{database_id}.{table_id} order by date'
# save result into dataframe
df_gcp = client.query(sql).to_dataframe().set_index('date')

In [8]:
df_gcp

,fed_rate
date,
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83
...,...
2024-05-01,5.33
2024-06-01,5.33
2024-07-01,5.33


In [9]:
# check if dataframes are the same before and after store in BigQuery
df.equals(df_gcp)

True